In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import random
from sqlalchemy import create_engine
from sqlalchemy import Table, Column, Integer, String, MetaData, ForeignKey, select
from sqlalchemy.sql import and_, or_, not_

In [ ]:
engine = create_engine('sqlite:///HoopStat.db')
conn = engine.connect()
metadata = MetaData(bind=None)
games = Table('games', metadata, autoload = True, autoload_with = engine)
events = Table('events', metadata, autoload = True, autoload_with = engine)

In [ ]:
stmt = select([games.c.id]).where(games.columns.isProcessed == True)
results = conn.execute(stmt).fetchall()
gameIDs = [value for value, in results]
gameIDs[0]

In [ ]:
stmt = "SELECT * FROM events WHERE gameid = '" + gameIDs[10] + "'"
n1 = 0
    
df = pd.read_sql_query(stmt,engine)
df['scoreHome'] = df['scoreHome'].map(int)
home = df[df['team']=='Home']
away = df[df['team']=='Away']

home['playScore'] = home.scoreHome - home.scoreHome.shift(1)
home.head(40)

In [ ]:
stmt = "SELECT * FROM events"
conn.execute(stmt).fetchall()

In [ ]:
stmt = "\
SELECT \
action\
,COUNT(1) \
FROM events \
GROUP BY \
action"

conn.execute(stmt).fetchall()

In [ ]:
stmt = "SELECT  \
e.gameid\
,e.period\
,e.time\
,e.action AS action1 \
,e2.action AS action2 \
,e.team AS team1 \
,e2.team AS team2 \
FROM events AS e \
INNER JOIN events AS e2 \
ON e.gameid = e2.gameid \
AND e.period = e2.period \
AND e.time = e2.time \
AND e.action != e2.action"

df = pd.read_sql_query(stmt,engine)

df.head(25)

In [ ]:
stmt = "DELETE FROM events"
conn.execute(stmt)

In [ ]:
stmt = games.update().values(isProcessed=False)
conn.execute(stmt)

In [ ]:
conn.close()